In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install polars

In [ ]:
import concurrent.futures
import math

from tqdm import tqdm
import numpy as np
import polars as pl
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine

# MRR@100

In [ ]:
train = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/train_task1.parquet")

In [ ]:
candidates = pl.concat([
    pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/interim/candidates/task1/similar_products_10_DE.parquet").sort(["locale", "item", "similarity_score"], descending=[False, False, True]).groupby(["locale", "item"], maintain_order=True).head(150),
    pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/interim/candidates/task1/similar_products_10_UK.parquet").sort(["locale", "item", "similarity_score"], descending=[False, False, True]).groupby(["locale", "item"], maintain_order=True).head(150),
    pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/interim/candidates/task1/similar_products_10_JP.parquet").sort(["locale", "item", "similarity_score"], descending=[False, False, True]).groupby(["locale", "item"], maintain_order=True).head(150),
])

In [ ]:
candidates = candidates.filter(pl.col("similarity_score") != 0)
candidates.write_parquet("/content/drive/MyDrive/kddcup2023/data/interim/candidates/task1/similar_products_13.parquet")

In [ ]:
# last_item
last_item_list = []
prev_items_list = train["prev_items"].to_list()
for prev_items in prev_items_list:
    last_item_list.append(prev_items[-1])
train = train.with_columns(pl.Series(name="last_item", values=last_item_list))

In [ ]:
train = train[["locale", "session_id", "last_item", "next_item"]]

In [ ]:
label_lists = []
n_rows = 500_000
for df in tqdm(train.iter_slices(n_rows=n_rows), total=math.ceil(train.height/n_rows)): # specify "total" parameter to display tqdm progress bar
    # process data
    df = df.join(candidates, left_on=["last_item", "locale"], right_on=["item", "locale"], how="left")
    df = df.with_columns((pl.col("candidate_item") == pl.col("next_item")).cast(pl.Int8).alias("label"))
    label_lists.extend(df.groupby("session_id", maintain_order=True).all()["label"].to_list())

100%|██████████| 7/7 [03:05<00:00, 26.53s/it]


In [ ]:
# MRR
rr = 0
for labels in label_lists:
    labels = labels[:100]
    for i, label in enumerate(labels):
        if label == 1:
            rr += 1 / (i+1)
            break
mrr = rr / len(label_lists)
print("MRR:", round(mrr, 5))

MRR: 0.20243
